This notebook shows how to do inference with yolonas and pretrained weights using the gpu. 

To add kernel I managed to add path to where venvs are stored, this solved the issue. 

Länk till YOLO NAS 2025
https://www.labellerr.com/blog/ultimate-yolo-nas-guide/ 

In [ ]:
# need to install these packages in the venv, don' know if this can be done in the notebook or must be done in the terminal
#!pip install ultralytics super_gradients opencv-python


# Important Note: 
#   * Use python<=3.11 as above python version donot support super_gradients
#   * onnx may be needed to reinstall with the correct versions needed by ultralytics, for me it solved the issue with importing onnxruntime



In [ ]:
import matplotlib.pyplot as plt
import cv2

In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="2sj781jgq3jIhtK8shSJ")
project = rf.workspace("abbrock-bolt-detection").project("yolo-detection-final-training")
version = project.version(6)
dataset = version.download("darknet")
                

In [ ]:
# Get gpu or cpu device
import torch
import os

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
torch.cuda.empty_cache()
print("Device: ", DEVICE)

MODEL_ARCH = "yolo_nas_m" 

# you can increase this number depending on your GPU, more batch size means faster training
BATCH_SIZE = 8 
# Epoch Number
MAX_EPOCHS = 300

# base directory
HOME=os.getcwd()
print("Home dir: ", HOME)
EXPERIMENT_NAME="detect_bolts_test_final_det_v6-1"
CHECKPOINT_DIR = "/import/bulkhome/c21ion/exjobb/yolonas_test_setup/checkpoints"

In [ ]:
import sys, os
print("sys.executable:", sys.executable)
print("python:", sys.version.splitlines()[0])
print("cwd:", os.getcwd())
print("sys.path[0]:", sys.path[0])
try:
    import onnxruntime as ort, inspect, sys
    print("module file:", getattr(ort, "__file__", None))
    print("version:", getattr(ort, "__version__", None))
    import onnxruntime.capi.onnxruntime_validation as v
    print("validation attrs:", [a for a in dir(v) if "package" in a.lower() or "version" in a.lower()])
except Exception as e:
    print("validation import error:", type(e).__name__, e)

In [ ]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

# Dataset , Label information
dataset_params = {
    'data_dir': HOME + '/YOLO-detection-final-training-6-yolov8', # path to dataset 
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'classes': ['Rockbolts'] # labels here 
}
print("Dataset params: ", dataset_params)

In [ ]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

In [ ]:
import inference
model = inference.load_roboflow_model("yolo-nas-l-640")


In [ ]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH,  # yolo_nas_m
    num_classes=len(dataset_params['classes']),
    #num_classes=None,
    pretrained_weights="coco", # TODO: Must remember to use weights with valid licence
    #checkpoint_path="/import/bulkhome/c21ion/exjobb/yolonas_test_setup/checkpoints/detect_bolts_test_1/RUN_20251020_082229_796307/ckpt_latest.pth"
)

In [ ]:
# Setup training params. 
# Demonstration says that 'mixed_precision' may be set to true, 
# but demo had problems due to its gpu
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,

    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision":  False , # TRUE BY DEFAULT , depending to GPU setting this to True might cause nan value problem in metrics
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300, # Should be lowered to 5-10
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000, # Should be lowered to 50-100
                max_predictions=300, # Should be lowered to around 5
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [ ]:
# Train the model
import numpy

torch.serialization.add_safe_globals([
    numpy._core.multiarray._reconstruct, 
    numpy.ndarray, 
    numpy.dtype,
    numpy.dtypes.Float64DType
])

trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

In [ ]:
# Load the best model from training
from super_gradients.training import models

inference_model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    checkpoint_path="/import/bulkhome/c21ion/exjobb/yolonas_test_setup/checkpoints/detect_bolts_test_1/RUN_20251020_102530_541823/ckpt_best.pth" # Select model
).to(DEVICE)

In [ ]:
# Get base model only without training head etc.
inference_model = model

Select if prediction or sample from training data should be presented by running either of the following two sections. 

In [ ]:
# Verify training data
import os
import numpy as np

# --- Output lists ---
NUM_TO_PRINT = 20
images_to_show = [] # format (image, (bboxes, confidences, labels))
# --- END Output lists---

val_data_label_path = os.path.join(dataset_params['data_dir'] ,dataset_params['val_labels_dir'])
val_data_image_path =  os.path.join(dataset_params['data_dir'] ,dataset_params['val_images_dir'])

folder_labels = os.listdir(val_data_label_path)

for _ in range(NUM_TO_PRINT):  # Show 5 random images, always assume there is only one bbox per image
    lines = []
    while len(lines) == 0:
        random_index = np.random.randint(0, len(folder_labels))

        label_name = folder_labels[random_index]
        image_name = label_name.split(".txt")[0] + ".jpg"
        print(f"Selected image: {image_name} with index {random_index}")

        image_path = os.path.join(val_data_image_path, image_name)
        label_path = os.path.join(val_data_label_path, label_name)
        image = cv2.imread(image_path)

        with open(label_path, "r") as f:
            lines = [l.strip() for l in f if l.strip()]


    line = lines[0]
    toks = line.split()
    print("Label line: ", line)
    if len(toks) < 5:
        print("Invalid label line: ", line)
    cls_id = toks[0]
    try:
        x_c = float(toks[1]); y_c = float(toks[2])
        bw = float(toks[3]); bh = float(toks[4])
    except ValueError:
        print("Invalid label line: ", line)
    print(f"Tokens: {toks}")

    x1 = int((x_c - bw/2) * 640)
    y1 = int((y_c - bh/2) * 640)
    x2 = int((x_c + bw/2) * 640)
    y2 = int((y_c + bh/2) * 640)    

    bbox = (x1, y1, x2, y2)
    confidence = 1.0
    label = int(cls_id)

    # Package in lists to comply with model.predict output
    images_to_show.append(((image, image_name), ([bbox], [confidence], [label])))
    


In [ ]:
# --- Perform prediction and prepare for display ---
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# Output
NUM_TO_PRINT = 5
images_to_show = [] # format (image, (bboxes, confidences, labels))

# Verify model prediction by taking random image from validation set
val_data_label_path = os.path.join(dataset_params['data_dir'] ,dataset_params['val_labels_dir'])
val_data_image_path =  os.path.join(dataset_params['data_dir'] ,dataset_params['val_images_dir'])

folder_labels = os.listdir(val_data_label_path)

for _ in range(NUM_TO_PRINT):  # Show 5 random images
    random_index = np.random.randint(0, len(folder_labels))

    label_name = folder_labels[random_index]
    image_name = label_name.split(".txt")[0] + ".jpg"
    print(f"Selected image: {image_name} with index {random_index}")

    image_path = os.path.join(val_data_image_path, image_name)
    label_path = os.path.join(val_data_label_path, label_name)
    image = cv2.imread(image_path)

    # --- predict ---
    model_result = inference_model.predict(image, conf=0.35)

    print(model_result.prediction)

    # Bounding boxes, labels, confidence, and label dictionary
    bboxes = model_result.prediction.bboxes_xyxy
    print("Bboxes: ", bboxes)
    confidences = model_result.prediction.confidence
    labels = model_result.prediction.labels

    images_to_show.append(((image, image_name), (bboxes, confidences, labels)))

In [ ]:
# Draw bounding boxes and labels on the image
label_dict={0:"Rockbolts"}

for (image, image_name), (bboxes, confidences, labels) in images_to_show:

    for i in range(len(bboxes)):
        bbox = bboxes[i]
        confidence = confidences[i]
        label = labels[i]
        print(f"Box: {bbox}, Confidence: {confidence}, Label: {label}")
        
        # Coordinates of the bounding box
        x1, y1, x2, y2 = [int(coord) for coord in bbox]
        

        # Draw the rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)

        cv2.putText(image, image_name, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                    fontScale=0.5, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)

        # Create label text with confidence
        label_text = f"{label}: {confidence:.2f}"

        # Put the label text above the bounding box
        cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                    fontScale=0.5, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)

    # Convert BGR to RGB for displaying in matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image using matplotlib
    plt.figure(figsize=(5, 5))
    plt.imshow(image_rgb)
    plt.axis('off')  # Turn off axis
    plt.show()

In [ ]:
#evalutate model performance

model.eval()